# Ejercicio: Maximizar Recall (Sensibilidad) en Cáncer de Mama

**Objetivo:** Crear un modelo de Regresión Logística que priorice **maximizar el Recall** para detectar tumores **Malignos**.

**El Problema:**
En este dataset, un **Falso Negativo (FN)** es el peor error posible: decirle a un paciente con un tumor maligno que es benigno. Para minimizar los FN, debemos maximizar el Recall.

**El Reto (Invertir Clases):**
Por defecto, `sklearn` etiqueta:
* `0 = Maligno`
* `1 = Benigno`

Si calculamos el Recall de la clase `1` (la positiva por defecto), estaríamos optimizando para encontrar tumores *benignos*, lo cual es inútil.

**Solución:**
1.  **Invertiremos las etiquetas:** `Maligno` se convertirá en `1` (nuestra clase positiva) y `Benigno` en `0` (la negativa).
2.  Ajustaremos el **umbral (threshold)** de decisión para asegurarnos de capturar tantos casos `1` (Malignos) como sea posible.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, PrecisionRecallDisplay

# Configuración de gráficos
#plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# 1. Cargar los datos
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

# Vemos las etiquetas originales:
print("--- Etiquetas Originales ---")
print(f"Nombres: {cancer.target_names}")
print(f"Datos (y): {np.bincount(y)}")
print("0 = Maligno, 1 = Benigno (¡No queremos esto!)")

# 2. Invertir las clases (¡El paso clave!)
# Queremos: 1 = Maligno, 0 = Benigno
# Usamos np.where: donde y sea 0, pon 1; donde sea 1, pon 0.
y_invertido = np.where(y == 0, 1, 0)

print("\n--- Etiquetas Invertidas ---")
print(f"Datos (y_invertido): {np.bincount(y_invertido)}")
print("0 = Benigno, 1 = Maligno (¡Correcto!)")

# 3. Escalar y dividir los datos (usando y_invertido)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled,
    y_invertido, # Usamos nuestras etiquetas invertidas
    test_size=0.3,
    random_state=42,
    stratify=y_invertido # Aseguramos proporción de malignos/benignos
)

--- Etiquetas Originales ---
Nombres: ['malignant' 'benign']
Datos (y): [212 357]
0 = Maligno, 1 = Benigno (¡No queremos esto!)

--- Etiquetas Invertidas ---
Datos (y_invertido): [357 212]
0 = Benigno, 1 = Maligno (¡Correcto!)


In [ ]:
# Entrenamos la Regresión Logística
# Aumentamos 'max_iter' para asegurar convergencia
modelo = LogisticRegression(max_iter=5000)
modelo.fit(X_train, y_train)

# Obtenemos las probabilidades para la CLASE 1 (Maligno)
# .predict_proba() devuelve [prob_clase_0, prob_clase_1]
y_probabilidades = modelo.predict_proba(X_test)[:, 1]


## Evaluación 1: Con Umbral por Defecto (0.5)

Veamos qué tan bueno es el modelo con el umbral estándar de 0.5.

In [ ]:
# .predict() usa el umbral 0.5 por defecto
y_pred_default = modelo.predict(X_test)

print("--- MÉTRICAS CON UMBRAL POR DEFECTO (0.5) ---")
print(classification_report(y_test, y_pred_default, target_names=['0: Benigno', '1: Maligno']))

print("\nMatriz de Confusión (Umbral 0.5):")
print("Filas: Realidad, Columnas: Predicción")
cm_default = confusion_matrix(y_test, y_pred_default)
print(cm_default)

# Análisis de Falsos Negativos
fn_default = cm_default[1, 0]
print(f"\n¡ALERTA! Falsos Negativos (FN) con umbral 0.5: {fn_default}")

--- MÉTRICAS CON UMBRAL POR DEFECTO (0.5) ---
              precision    recall  f1-score   support

  0: Benigno       0.96      0.99      0.98       107
  1: Maligno       0.98      0.94      0.96        64

    accuracy                           0.97       171
   macro avg       0.97      0.96      0.97       171
weighted avg       0.97      0.97      0.97       171


Matriz de Confusión (Umbral 0.5):
Filas: Realidad, Columnas: Predicción
[[106   1]
 [  4  60]]

¡ALERTA! Falsos Negativos (FN) con umbral 0.5: 4


**Análisis (Umbral 0.5):**

Observa el reporte de `1: Maligno`. Verás un **Recall** alto (quizás ~0.95), pero no perfecto.

La matriz de confusión mostrará un número pequeño (pero > 0) en la esquina inferior izquierda. Esos son los **Falsos Negativos**: pacientes con tumores malignos que el modelo etiquetó como benignos. ¡Es inaceptable!

## Evaluación 2: Ajustar Umbral para Maximizar Recall (ej. 0.1)

Vamos a bajar el umbral drásticamente. Le diremos al modelo: "Si estás *mínimamente* seguro (ej. > 10% de probabilidad) de que es maligno, márcalo como maligno".

In [ ]:
# CAMBIA EL NUEVO UMBRAL(modificando el umbral por defecto o inicial)
nuevo_umbral = 0.5

# Creamos las predicciones manualmente
y_pred_nuevo = (y_probabilidades > nuevo_umbral).astype(int)

print(f"--- MÉTRICAS CON UMBRAL AJUSTADO ({nuevo_umbral}) ---")
print(classification_report(y_test, y_pred_nuevo, target_names=['0: Benigno', '1: Maligno']))

print(f"\nMatriz de Confusión (Umbral {nuevo_umbral}):")
print("Filas: Realidad, Columnas: Predicción")
cm_nuevo = confusion_matrix(y_test, y_pred_nuevo)
print(cm_nuevo)

# Análisis de Falsos Negativos
fn_nuevo = cm_nuevo[1, 0]
print(f"\n¡ÉXITO! Falsos Negativos (FN) con umbral {nuevo_umbral}: {fn_nuevo}")

--- MÉTRICAS CON UMBRAL AJUSTADO (5) ---
              precision    recall  f1-score   support

  0: Benigno       0.63      1.00      0.77       107
  1: Maligno       0.00      0.00      0.00        64

    accuracy                           0.63       171
   macro avg       0.31      0.50      0.38       171
weighted avg       0.39      0.63      0.48       171


Matriz de Confusión (Umbral 5):
Filas: Realidad, Columnas: Predicción
[[107   0]
 [ 64   0]]

¡ÉXITO! Falsos Negativos (FN) con umbral 5: 64


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Análisis Final**

¡Mira los resultados!

1.  **Recall (para `1: Maligno`) HA SUBIDO:** Debería ser `1.00` (o 100%). ¡Hemos encontrado a *todos* los pacientes malignos! El número de Falsos Negativos (esquina inferior izquierda) es ahora **0**.

2.  **Precision (para `1: Maligno`) HA BAJADO:** Este es el "costo". Para estar seguros, el modelo ahora genera más Falsos Positivos (FP) (esquina superior derecha). Estos son pacientes con tumores *benignos* que fueron etiquetados como *malignos*.

